In [3]:
import pandas as pd
import numpy as np
from get_ap_data import get_wifi_aps_pywifi_windows_robust_v2
from get_longlat_from_ps import get_location_from_powershell

In [4]:
def get_wifi_data():
    spot = input("Masukkan nama spot: ")
    # filename = input("Masukkan nama file untuk menyimpan data (tanpa ekstensi): ")
    spot_df = pd.DataFrame({
        "spot": [spot],
    })
    script_path = "get_location.ps1" # Ganti dengan path skrip Anda
    detected_aps_df = get_wifi_aps_pywifi_windows_robust_v2()
    location_df = get_location_from_powershell(script_path)

    detected_aps_df = detected_aps_df[detected_aps_df['ssid'].str.strip().str.lower() == 'untirta']
    location_df_final = pd.DataFrame(np.tile(location_df.values, (detected_aps_df.shape[0], 1)), columns=location_df.columns).iloc[:detected_aps_df.shape[0]]
    spot_df_final = pd.DataFrame(np.tile(spot_df.values, (detected_aps_df.shape[0], 1)), columns=spot_df.columns).iloc[:detected_aps_df.shape[0]]
    merged_dfl = pd.concat([detected_aps_df, location_df_final, spot_df_final], axis=1)
    merged_dfl = merged_dfl.drop_duplicates(subset=['bssid', 'signal_dBm', 'frequency_MHz'])
    print(f"{len(merged_dfl)} Jaringan Untirta Terdeteksi")
    return merged_dfl


In [ ]:
loc_1 = get_wifi_data()
print("Data WiFi berhasil dikumpulkan")

Peringatan: pywifi.const.AKM_TYPE_NAMES tidak ditemukan. Menggunakan fallback.
Peringatan: pywifi.const.CIPHER_TYPE_NAMES tidak ditemukan. Menggunakan fallback.
Peringatan: pywifi.const.AUTH_ALG_NAMES tidak ditemukan. Menggunakan fallback.
Menggunakan antarmuka: MediaTek Wi-Fi 6 MT7921 Wireless LAN Card
Memulai pemindaian jaringan WiFi...
Menunggu 8 detik hingga pemindaian selesai...
6 Jaringan Untirta Terdeteksi
Data WiFi berhasil dikumpulkan


In [ ]:
loc_1 

,ssid,bssid,signal_dBm,frequency_MHz,akm_type,cipher_type,auth_alg,latitude,longitude,spot
0,UNTIRTA,9e:2b:a6:8e:5e:6e:,-68.0,2412000.0,AKM_TYPE_WPA2PSK,CIPHER_TYPE_NONE,AUTH_ALG_OPEN,-5.996275,106.030654,
1,UNTIRTA,98:4a:6b:94:ab:ff:,-63.0,2462000.0,AKM_TYPE_WPA2PSK,CIPHER_TYPE_NONE,AUTH_ALG_OPEN,-5.996275,106.030654,
2,UNTIRTA,9a:4a:6b:84:ab:ff:,-53.0,5765000.0,AKM_TYPE_WPA2PSK,CIPHER_TYPE_NONE,AUTH_ALG_OPEN,-5.996275,106.030654,
3,UNTIRTA,9c:2b:a6:be:5e:6e:,-50.0,5765000.0,AKM_TYPE_WPA2PSK,CIPHER_TYPE_NONE,AUTH_ALG_OPEN,-5.996275,106.030654,
4,UNTIRTA,9a:4a:6b:84:8b:b6:,-78.0,5745000.0,AKM_TYPE_WPA2PSK,CIPHER_TYPE_NONE,AUTH_ALG_OPEN,-5.996275,106.030654,
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.996275,106.030654,


In [1]:
import time
import threading
import math
import random
import subprocess
import platform
import pandas as pd
import numpy as np
import os
import re
from queue import Queue
import pywifi
from pywifi import const

In [2]:
# =========================
# KONFIGURASI SIMULASI
# =========================
USE_SIMULATION = False  # Ubah ke False untuk menggunakan PowerShell GPS
SIMULATION_STEP = 50   # meter, setiap langkah simulasi
SCAN_INTERVAL = 10     # detik antar pemindaian
START_POSITION = (0.0, 0.0)  # koordinat awal simulasi

# =========================
# Fallback jika pywifi const tidak tersedia
# =========================
MANUAL_AKM_TYPE_NAMES = {
    0: "AKM_TYPE_NONE", 1: "AKM_TYPE_WPA", 2: "AKM_TYPE_WPAPSK",
    3: "AKM_TYPE_WPA2", 4: "AKM_TYPE_WPA2PSK", 5: "AKM_TYPE_UNKNOWN",
    6: "AKM_TYPE_WPA3", 7: "AKM_TYPE_WPA3PSK", 8: "AKM_TYPE_WPA3_SAE",
}
MANUAL_CIPHER_TYPE_NAMES = {
    0: "CIPHER_TYPE_NONE", 1: "CIPHER_TYPE_WEP", 2: "CIPHER_TYPE_TKIP",
    3: "CIPHER_TYPE_CCMP", 4: "CIPHER_TYPE_UNKNOWN", 5: "CIPHER_TYPE_GCMP128",
    6: "CIPHER_TYPE_GCMP256",
}
MANUAL_AUTH_ALG_NAMES = {
    0: "AUTH_ALG_OPEN", 1: "AUTH_ALG_SHARED", 2: "AUTH_ALG_LEAP",
}

# =========================
# Fungsi bantu: Haversine Distance
# =========================
def haversine(coord1, coord2):
    R = 6371000
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [3]:
def get_current_location():
    if USE_SIMULATION:
        lat = START_POSITION[0] + random.uniform(-0.0005, 0.0005)
        lon = START_POSITION[1] + random.uniform(-0.0005, 0.0005)
        return (lat, lon)
    else:
        df = get_location_from_powershell("get_location.ps1")
        if not df.empty:
            return df.iloc[0]['latitude'], df.iloc[0]['longitude']
        return START_POSITION

# =========================
# Ambil Lokasi dari PowerShell
# =========================
def get_location_from_powershell(script_name):
    script_path = os.path.join(os.getcwd(), script_name)
    try:
        command = ["powershell.exe", "-ExecutionPolicy", "Bypass", "-File", script_path]
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        output_lines = result.stdout.strip().split('\n')
        location_data = next((line for line in output_lines if "Latitude:" in line and "Longitude:" in line), None)
        if location_data:
            match = re.search(r"Latitude:\s*(-?\d+\.\d+),\s*Longitude:\s*(-?\d+\.\d+)", location_data)
            if match:
                lat = float(match.group(1))
                lon = float(match.group(2))
                return pd.DataFrame({"latitude": [lat], "longitude": [lon]})
        return pd.DataFrame()
    except Exception as e:
        print(f"[Lokasi Error] {e}")
        return pd.DataFrame()

# =========================
# Scan Wi-Fi robust
# =========================
def get_wifi_aps_pywifi_windows_robust_v2():
    akm_names = getattr(const, 'AKM_TYPE_NAMES', MANUAL_AKM_TYPE_NAMES)
    cipher_names = getattr(const, 'CIPHER_TYPE_NAMES', MANUAL_CIPHER_TYPE_NAMES)
    auth_alg_names = getattr(const, 'AUTH_ALG_NAMES', MANUAL_AUTH_ALG_NAMES)

    try:
        wifi = pywifi.PyWiFi()
        iface = wifi.interfaces()[0]
        iface.scan()
        time.sleep(8)
        scan_results = iface.scan_results()
        aps_data = []
        for profile in scan_results:
            ssid = profile.ssid.encode('raw_unicode_escape').decode('utf-8', 'ignore')
            akm_str = ", ".join([akm_names.get(a, str(a)) for a in profile.akm]) if isinstance(profile.akm, list) else akm_names.get(profile.akm, str(profile.akm))
            cipher_str = cipher_names.get(profile.cipher, str(profile.cipher))
            auth_str = auth_alg_names.get(profile.auth, str(profile.auth))
            aps_data.append({
                "ssid": ssid,
                "bssid": profile.bssid,
                "signal_dBm": profile.signal,
                "frequency_MHz": profile.freq,
                "akm_type": akm_str,
                "cipher_type": cipher_str,
                "auth_alg": auth_str
            })
        return pd.DataFrame(aps_data) if aps_data else pd.DataFrame()
    except Exception as e:
        print(f"[WiFi Error] {e}")
        return pd.DataFrame()

# =========================
# Ambil data snapshot
# =========================
def get_wifi_data(spot=None, script_path="get_location.ps1"):
    if spot is None:
        spot = input("Masukkan nama spot: ")
    spot_df = pd.DataFrame({"spot": [spot]})
    detected_aps_df = get_wifi_aps_pywifi_windows_robust_v2()
    location_df = get_location_from_powershell(script_path)
    detected_aps_df = detected_aps_df[detected_aps_df['ssid'].str.strip().str.lower() == 'untirta']
    if detected_aps_df.empty or location_df.empty:
        print("Data tidak lengkap atau tidak ada jaringan UNTIRTA terdeteksi.")
        return pd.DataFrame()
    location_df_final = pd.DataFrame(np.tile(location_df.values, (detected_aps_df.shape[0], 1)), columns=location_df.columns)
    spot_df_final = pd.DataFrame(np.tile(spot_df.values, (detected_aps_df.shape[0], 1)), columns=spot_df.columns)
    merged_dfl = pd.concat([detected_aps_df, location_df_final, spot_df_final], axis=1)
    merged_dfl = merged_dfl.drop_duplicates(subset=['bssid', 'signal_dBm', 'frequency_MHz'])
    print(f"{len(merged_dfl)} Jaringan Untirta Terdeteksi")
    return merged_dfl

# =========================
# Prompt CLI Interaktif
# =========================
def interactive_prompt():
    print("\nKetik 'ambil snapshot' untuk menyimpan kondisi jaringan.")
    print("Ketik 'lanjut' untuk melanjutkan pemindaian.")
    print("Ketik 'keluar' untuk mengakhiri.\n")
    return input("Perintah: ")

# =========================
# Fungsi Placeholder
# =========================
def custom_main_function():
    print("[INFO] Fungsi utama khusus bisa ditambahkan di sini nanti.")


In [5]:
# =========================
# Loop Utama
# =========================
def main_loop():
    print("Memulai pemindaian bergerak setiap 50 meter...")
    prev_location = get_current_location()
    log_data = []
    while True:
        time.sleep(SCAN_INTERVAL)
        curr_location = get_current_location()
        distance = haversine(prev_location, curr_location)
        print(f"Berpindah sejauh {distance:.2f} meter.")
        if distance >= 50:
            print("> Ambil snapshot otomatis karena jarak > 50m")
            df = get_wifi_data(spot="auto_loop")
            if not df.empty:
                log_data.append(df)
            prev_location = curr_location
        user_cmd = interactive_prompt().strip().lower()
        if user_cmd == "ambil snapshot":
            df = get_wifi_data()
            if not df.empty:
                log_data.append(df)
        elif user_cmd == "lanjut":
            continue
        elif user_cmd == "keluar":
            print("Program dihentikan. Menyimpan data jika ada...")
            if log_data:
                final_log = pd.concat(log_data, ignore_index=True)
                final_log.to_csv("log_wifi_snapshot.csv", index=False)
                print("Data disimpan ke 'log_wifi_snapshot.csv'")
            break
        else:
            print("Perintah tidak dikenal.")

# =========================
# Eksekusi
# =========================
if __name__ == "__main__":
    main_loop()

Memulai pemindaian bergerak setiap 50 meter...
[Lokasi Error] Command '['powershell.exe', '-ExecutionPolicy', 'Bypass', '-File', 'c:\\Users\\azizl\\OneDrive\\Documents\\Folder Semester 6\\Machine Learning\\Project UAS\\Auto Detection IP\\get_location.ps1']' returned non-zero exit status 1.
Berpindah sejauh 11780057.93 meter.
> Ambil snapshot otomatis karena jarak > 50m


KeyboardInterrupt: 